In [2]:
# !pip install river
# !pip install microprediction
# !pip install numpy
import numpy
from river import linear_model
from river import preprocessing
from river import neighbors
from river import neural_net as nn
from river import metrics
from river import optim

C:\Users\This PC\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
# write_key = '7248d354a9e2a4aac6feb7dcc433acb1'

In [5]:
from microprediction import MicroCrawler
from microprediction import MicroReader
import math 
import numpy as np
import river.stats
import collections
import functools

reader = MicroReader()

Var: 0.

In [22]:
class MyCrawler(MicroCrawler):
    def __init__(self, **kwargs):
        """
        
        """
        
        super().__init__(**kwargs)
        self.freq = {}
        self.var = {}
        # dictionary saving models for datastream
        self.model = {}
        self.queue = {}
        self.names = set()
        self.count = {}
        # Create 10 most used models for regression problems
        model = preprocessing.StandardScaler()
        contents = [linear_model.BayesianLinearRegression(), 
                    nn.MLPRegressor(hidden_dims=(5,),activations=(nn.activations.ReLU,nn.activations.ReLU,nn.activations.Identity),optimizer=optim.SGD(1e-3),seed=42)]
        learning_rate = [0.01, 0.001, 0.0001, 0.1, 0.00001, 0.02, 0.2, 0.05] 
        for i in learning_rate:
            contents.append(linear_model.LinearRegression(optimizer=optim.SGD(lr=i)))

        models = [model | content for content in contents]
        # 10 models to be chosen from
        self.models = models
        self.init_models = {}

        for model in models:
            self.count[model] = 0
            
        self.evaluate = {}
        self.metric = {}
        self.init_val = {}
        self.init_model = {}
           
    def get_freq(self, frequency):
        if frequency >= 86400:
        # The frequency is bigger than a day, then seasonality should be weekly
            return math.ceil(604800/frequency)
        elif frequency >= 3600:
        # The frequency is bigger than an hour but less tha a day, then seasonality should be hourly
            return math.ceil(86400/frequency)
        elif frequency >= 60:
        # The frequency is bigger than a minute but less than an hour, then seasonality should be minute
            return math.ceil(3600/frequency)
        else:
        # The frequency is less than a minute, then seasonality is second
            return math.ceil(60/frequency)

    def number_of_element_in_season(self, lagged_time_array):
        if len(lagged_time_array) == 1:
            return self.get_freq(lagged_time_array[0])
        else:
            return self.get_freq(lagged_time_array[0] - lagged_time_array[1])
    
        
    def sample(self, lagged_values, lagged_times=None, name=None, delay=None, **ignored):
        """ An example of a sample method. This is where all the intelligence goes
               :param lagged_values [ float ]    List with most recent listed first
               :param lagged_times [ float ]    List of epoch times, most recent listed first
               :param name          str         Name of stream
               :param delay         int         Prediction horizon in seconds
               **ignored                        For future compatability.
               :returns            [ float ]    A vector of numbers somewhat indicative of a probability density, of length self.num_predictions
        """
        if name not in self.names:
            self.names.add(name)
            self.evaluate[name] = [0 for _ in range (10)]
            self.metric[name] = [metrics.R2() for _ in range (10)]
            self.init_val[name] = [[] for _ in range (10)]
            self.init_model[name] = self.models
            self.var[name] = river.stats.Var()

        if name not in self.models:
            self.freq[name] = self.number_of_element_in_season(lagged_times)
            self.queue[name] = collections.deque(maxlen=self.freq[name])
            for i in range (len(self.models)):
            # Warm start
                for x, y in zip(reversed(lagged_times), reversed(lagged_values)):
                    new_feats = {f"lag_{i}": v for i, v in enumerate(self.queue[name])}

                    if len(new_feats) < self.freq[name]:
                        continue

                    # copy of x
                    x_ = {'time': x}
                    x_.update(new_feats)

                    y_pred = self.init_model[name][i].predict_one(x_)
                    self.init_model[name][i].learn_one(x_, y)

                    self.init_val[name][i].append((y, y_pred))
                    self.metric[name][i].update(y, y_pred)
                    self.queue[name].append(y)
                    variance = self.var[name].update(y_pred).get()
    
            self.evaluate[name] = [metric.get() for metric in self.metric[name]]
            temp = -1e9
            t = 0
            for i in range (10):
                if self.evaluate[name][i] > temp:
                    t = i
                    temp = self.evaluate[name][i]
            self.model[name] = self.models[t]

        x = {f"lag_{i}": v for i, v in enumerate(self.queue[name])}
        x["time"] = lagged_times[0]
        y = lagged_values[0]
        self.model[name].learn_one(x, y)
        self.queue[name].append(y)
        x = {f"lag_{i}": v for i, v in enumerate(self.queue[name])}
        x['time'] = 2*lagged_times[0] - lagged_times[1]
        y_pred = self.model[name].predict_one(x)
        variance = self.var[name].update(y_pred).get()

        return np.random.normal(y_pred, variance**0.5, 225)

In [ ]:
mw = MyCrawler(write_key = '7248d354a9e2a4aac6feb7dcc433acb1')
mw.run()

Gogo Coyote restarting at 2022-10-03 01:31:50.514560
('{"quietude": 50, "stop_loss": 10, "min_lags": 25, "num_active": 15, '
 '"num_pending_cancellations": 0, "current_balance": -794.6166240852173, '
 '"recent_errors": [], "currently_worst": [], "pending_cancellations": [], '
 '"upcoming": [["70::yarx_dgx.json", -1.1], ["70::yarx_algn.json", -0.1], '
 '["70::xray_162.json", 0.8]]}')
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::yarx_dgx.json', -1.7),
 ('70::yarx_algn.json', -0.7),
 ('70::xray_162.json', 0.2),
 ('910::yarx_vac.json', 5.2),
 ('70::noaa_wind_direction_46061.json', 6.2)]
('{"quietude": 50, "stop_loss": 10, "min_lags": 25, "num_active": 15, '
 '"num_pending_cancellations": 0, "current_balance": -794.6166240852173, '
 '"recent_errors": [], "currently_worst": [], "pending_cancellations": [], '
 '"upcoming": [["70::yarx_dgx.json", -3.0], ["70::yarx_algn.json", -2.0], '
 '[

Upcoming horizons and seconds to go ...
[('70::z2~noaa_wind_direction_46077~noaa_wind_speed_46073~70.json', 27.0),
 ('70::yarx_dgx.json', 47.7),
 ('70::yarx_algn.json', 48.3),
 ('70::xray_162.json', 48.6),
 ('70::noaa_wind_direction_46061.json', 49.9)]
Downtime for 16.9s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::yarx_dgx.json', 17.4),
 ('70::yarx_algn.json', 17.9),
 ('70::xray_162.json', 18.2),
 ('70::noaa_wind_direction_46061.json', 19.5),
 ('70::xray_601.json', 20.8)]
Downtime for 1.6s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z3~noaa_wind_direction_41008~noaa_wind_speed_41008~noaa_wind_speed_44005~3555.json',
  2.0),
 ('70::c2_rebalanced_80_mean.json', 5.5),
 ('70::z1~naafg_6~3555.json', 6.8),
 ('310::xray_312.json', 24.3),
 ('70::z2~noaa_wind_direction_46077~noaa_wi

Downtime for 13.3s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z2~noaa_wind_direction_46077~noaa_wind_speed_46073~70.json', 13.8),
 ('70::z1~tour_10~3555.json', 26.2),
 ('70::yarx_dgx.json', 34.5),
 ('70::yarx_algn.json', 35.1),
 ('70::xray_162.json', 35.4)]
Downtime for 8.6s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z1~tour_10~3555.json', 9.1),
 ('70::yarx_dgx.json', 17.4),
 ('70::yarx_algn.json', 18.0),
 ('70::xray_162.json', 18.3),
 ('70::noaa_wind_direction_46061.json', 19.1)]
Downtime for 4.6s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::yarx_dgx.json', 5.1),
 ('70::yarx_algn.json', 5.7),
 ('70::xray_162.json', 6.0),
 ('70::noaa_wind_direction_46061.json', 6.8),

Downtime for 8.2s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('310::z1~gnaaf_33333~3555.json', 8.6),
 ('70::z1~tour_10~3555.json', 8.7),
 ('70::yarx_dgx.json', 17.4),
 ('70::yarx_algn.json', 17.9),
 ('70::xray_162.json', 18.3)]
Downtime for 4.8s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::yarx_dgx.json', 5.3),
 ('70::yarx_algn.json', 5.8),
 ('70::xray_162.json', 6.2),
 ('70::noaa_wind_direction_46061.json', 6.6),
 ('70::xray_601.json', 8.9)]
Downtime for 1.7999999999999998s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z3~noaa_wind_direction_41008~noaa_wind_speed_41008~noaa_wind_speed_44005~3555.json',
  2.2),
 ('70::z1~naafg_6~3555.json', 6.1),
 ('70::z2~noaa_wind_direction

Upcoming horizons and seconds to go ...
[('70::z1~tour_10~3555.json', 4.0),
 ('70::yarx_dgx.json', 12.8),
 ('310::z1~tour_6_bad~3555.json', 13.2),
 ('70::yarx_algn.json', 13.4),
 ('70::xray_162.json', 13.7)]
Downtime for 5.1s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::yarx_dgx.json', 5.5),
 ('310::z1~tour_6_bad~3555.json', 5.9),
 ('70::yarx_algn.json', 6.1),
 ('70::xray_162.json', 6.4),
 ('70::noaa_wind_direction_46061.json', 6.7)]
Downtime for 2.0s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z3~noaa_wind_direction_41008~noaa_wind_speed_41008~noaa_wind_speed_44005~3555.json',
  2.4),
 ('70::z1~naafg_6~3555.json', 5.9),
 ('310::z1~gnaaf_33333~3555.json', 8.7),
 ('70::z1~xray_361~70.json', 29.2),
 ('70::z2~noaa_wind_direction_46077~noaa_wind_speed_46073~70.json', 37.8)]
Down

Downtime for 5.2s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z2~noaa_wind_direction_46077~noaa_wind_speed_46073~70.json', 5.6),
 ('70::finance-futures-heating_oil-log-return.json', 7.0),
 ('70::z1~tour_10~3555.json', 17.2),
 ('70::yarx_dgx.json', 26.3),
 ('70::yarx_algn.json', 26.9)]
Downtime for 6.4s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('70::z1~tour_10~3555.json', 6.9),
 ('70::yarx_dgx.json', 15.9),
 ('70::yarx_algn.json', 16.5),
 ('70::xray_162.json', 16.8),
 ('70::noaa_wind_direction_46061.json', 17.0)]
-------------------------------------------------------------
Just a reminder ... 
Your write key is 7248d354a9e2a4aac6feb7dcc433acb1
Your public key is 9090c0407e082083fc2dc0b2752e5775
Your nom de plume is Gogo Coyote
Put your write key in the dashboard at www.micropr

Upcoming horizons and seconds to go ...
[('70::z2~noaa_wind_direction_46077~noaa_wind_speed_46073~70.json', 1.9),
 ('70::finance-futures-heating_oil-log-return.json', 3.4),
 ('910::yarx_pri.json', 11.6),
 ('70::z1~tour_10~3555.json', 13.1),
 ('910::yarx_vac.json', 17.8)]
Downtime for 4.4s
Gogo Coyote has active submissions for 15 horizons (15 with no withdrawal request pending since restart)
Upcoming horizons and seconds to go ...
[('910::yarx_pri.json', 4.8),
 ('70::z1~tour_10~3555.json', 6.3),
 ('910::yarx_vac.json', 11.1),
 ('70::yarx_dgx.json', 15.6),
 ('70::yarx_algn.json', 16.4)]
Currently predicting for 15 horizons but found 4638 candidate streams to examine.
Submitted to yarx_tmus.json 70s horizon, and will do so again in 2411 seconds.
Submitted to horizon 70::yarx_tmus.json
-------------------------------------------------------------
Just a reminder ... 
Your write key is 7248d354a9e2a4aac6feb7dcc433acb1
Your public key is 9090c0407e082083fc2dc0b2752e5775
Your nom de plume is